In [3]:
import get_import
import get_db

# Acquire data

In [4]:
df = get_db.get_titanic_data()
df.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,None,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,None,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,None,Southampton,1


# Prepare

#### Check for nulls, duplicates, fill in the blanks, encode strings for decision tree, split data.

In [5]:
df.isna().sum()

passenger_id      0
survived          0
pclass            0
sex               0
age             177
sibsp             0
parch             0
fare              0
embarked          2
class             0
deck            688
embark_town       2
alone             0
dtype: int64

In [6]:
#drop columns
df = df.drop(columns=["deck", "class", "embarked"])

In [7]:
#set index
df = df.set_index("passenger_id")

In [8]:
#fill in the 2 null embark_town values with the mode of listed towns 
df.embark_town = df.embark_town.fillna(value=df.embark_town.mode())

In [9]:
#fill in the age nulls with the average age
df.age = df.age.fillna(value=df.age.median())

# Clean data

# Explore

# Evaluate

# Model

In [ ]:
df.shape

In [ ]:
df = df.drop_duplicates()
df.shape

In [ ]:
df = df.drop(columns=['deck', 'embarked', 'class', 'age'])
df.shape

In [ ]:
df['embark_town'] = df.embark_town.fillna(value='Southampton')

In [ ]:
dummy_df = pd.get_dummies(df[['sex', 'embark_town']], dummy_na = False, drop_first=[True, True])

In [ ]:
df.head()

In [ ]:
df = pd.concat([df, dummy_df], axis=1)
df.head()

In [ ]:
df.shape

In [ ]:
df = df.drop(columns=['sex','embark_town'])
df.shape

### Train, Split, Test

In [ ]:
def train_validate_test_split(df, target = 'survived', seed=123):
    '''
    This function takes in a dataframe, the name of the target variable
    (for stratification purposes), and an integer for a setting a seed
    and splits the data into train, validate and test. 
    Test is 20% of the original dataset, validate is .30*.80= 24% of the 
    original dataset, and train is .70*.80= 56% of the original dataset. 
    The function returns, in this order, train, validate and test dataframes. 
    '''
    train_validate, test = train_test_split(df, test_size=0.2, 
                                            random_state=seed, 
                                            stratify=df[target])
    train, validate = train_test_split(train_validate, test_size=0.3, 
                                       random_state=seed,
                                       stratify=train_validate[target])
    return train, validate, test


In [ ]:
# split into train, validate, test
train, validate, test = train_validate_test_split(df, target='survived', seed=123)

# create X & y version of train, where y is a series with just the target variable and X are all the features. 

X_train = train.drop(columns=['survived'])
y_train = train.survived

X_validate = validate.drop(columns=['survived'])
y_validate = validate.survived

X_test = test.drop(columns=['survived'])
y_test = test.survived

## Model

In [ ]:
# 1. Create the object
model = DummyClassifier(strategy='constant', constant=1) # here, we're predicting everyone survives
# 2. Fit the object
model.fit(X_train, y_train)

In [ ]:
accuracy = round(model.score(X_train, y_train), 2)
print(f'Training accuracy: {accuracy}')


In [ ]:
# TODO: view the accuracy on the validate split
model.score(X_validate, y_validate)

In [ ]:
model.predict(X_validate)

In [ ]:
# TODO: create a new column on the train dataframe that contains the models predictions
train['prediction'] = model.predict(X_train)

In [ ]:
train.head()

In [ ]:
# use the column you just created and the actual values in the survived column
# to generate a classification report
print(classification_report(train.survived, train.prediction, zero_division=True))

In [ ]:
pd.DataFrame(classification_report(train.survived, train.prediction, output_dict=True)).T

In [ ]:
# for classification you can change the algorithm to gini or entropy (information gain).  
# Default is gini.

clf = DecisionTreeClassifier(max_depth=3, random_state=123)

In [ ]:
# model.fit(X, y)

clf = clf.fit(X_train, y_train)

In [ ]:
import graphviz
from graphviz import Graph

dot_data = export_graphviz(clf, feature_names= X_train.columns, class_names=clf.classes_, rounded=True, filled=True, out_file=None)
graph = graphviz.Source(dot_data) 

graph.render('titanic_decision_tree', view=True)

### 1. What is your baseline prediction? 

### 1.a. What is your baseline accuracy? 

### remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). 

### 1.b. When you make those predictions, what is your accuracy? This is your baseline accuracy.

### 2. Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

### 3. Evaluate your in-sample results using the model score, confusion matrix, and classification report.

### 4. Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

### 5. Run through steps 2-4 using a different max_depth value.

### 6. Which model performs better on your in-sample data?

### 7. Which model performs best on your out-of-sample data, the validate set?